In [88]:
import sys
sys.path.append('..')
from preporcessing import TorchStandardScaler
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import random_split
from dataloader import *
import torch.nn as nn

In [69]:
dfX = pd.read_pickle('../data/processed/meanX.pkl').to_numpy()
dfy = pd.read_pickle('../data/processed/meanY.pkl').to_numpy()

dfX_torch = torch.from_numpy(dfX.astype(np.float32))
dfX_torch.size(0)


14569

In [54]:
scaler  = TorchStandardScaler()
scaler.fit(dfX_torch)
scaler.mean, scaler.std

(tensor([[ 7.1916e-01, -6.2164e-01,  4.9645e-01,  1.4198e+06, -4.9113e+04,
          -8.0033e+03, -4.0135e+02,  7.5098e+04,  1.3036e+05, -1.1548e+04,
           4.0135e+04]]),
 tensor([[3.3085e+00, 3.4537e+00, 2.6056e+00, 3.3296e+04, 1.6937e+05, 9.8403e+04,
          7.5670e+01, 5.9771e+04, 2.5072e+04, 4.6858e+04, 9.6293e+03]]))

In [87]:
train_idx,test_idx,val_idx=random_split(dfX_torch, [0.49,0.3,0.21], generator=torch.Generator().manual_seed(1))
len(train_idx.indices)

7139

In [89]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, fc_layers, device, dropout=0) -> None:
        super(LSTM, self).__init__() 
        self.device=device
        self.hidden_size = hidden_size
        self.num_layers =  num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,batch_first =True, dropout=dropout)
        self.activation = nn.ReLU()
        
        self.fc = nn.ModuleList()
        for layer in fc_layers:
            self.fc.append(nn.Linear(hidden_size,layer))
            self.fc.append(nn.BatchNorm1d(layer))
            self.fc.append(nn.ReLU())
        self.fc.append(fc_layers[-1],1)

       
    def forward(self,x):
        h0  =  torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device).requires_grad_()
        c0  =  torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device).requires_grad_()

        out, (hout,_) =  self.lstm(x,(h0,c0))
        #out = self.activation(out[:,-1,:])
        if self.debug:
            print(f"out= {out[:,-1,:]}, {hout.shape},hidden= {hout.view(-1, self.hidden_size)}")

        for layers in self.fc:
            out = self.fc(out)
        #out = self.fc(out)
        return out